In [ ]:
import gdown
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pprint
import random
import pandas as pd
import pulp
import itertools
import requests
import random
from geopy.distance import geodesic
import time
import pandas as pd
import os

Downloading...
From (original): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf
From (redirected): https://drive.google.com/uc?id=1zq17RENYLU1qrE2gNPsxRAPoysWMZAqf&confirm=t&uuid=2870eee6-d24f-4115-b21c-2d598873d27d
To: /content/mpsis_setup.py
100%|██████████| 12.1k/12.1k [00:00<00:00, 7.28MB/s]


In [ ]:
def generate_random_point(sw_lat, sw_lng, ne_lat, ne_lng):
    lat = random.uniform(sw_lat, ne_lat)
    lng = random.uniform(sw_lng, ne_lng)
    return lat, lng

def snap_to_road(lat, lng, api_key):
    url = f'https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    if 'snappedPoints' in data and len(data['snappedPoints']) > 0:
        snapped_point = data['snappedPoints'][0]['location']
        return snapped_point['latitude'], snapped_point['longitude']
    return None

def get_road_distance_time(origin, destination, api_key):
    origin_str = f'{origin[0]}, {origin[1]}'
    destination_str = f'{destination[0]}, {destination[1]}'
    departure_time = int(time.time()) + 24 * 3600
    url = (
        f'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins={origin_str}'
        f'&destinations={destination_str}&mode=driving&departure_time={departure_time}&key={api_key}'
    )
    response = requests.get(url)
    data = response.json()

    if data['status'] == 'OK':
        element = data['rows'][0]['elements'][0]
        if element['status'] == 'OK':
            distance_text = element['distance']['text']
            distance = float(distance_text.replace(' km', '').replace(',', ''))

            duration_text = element['duration']['text']
            duration_parts = duration_text.split()
            duration = int(duration_parts[0])
            if "hour" in duration_text:
                duration = int(duration_parts[0]) * 60 + int(duration_parts[2])

            return distance, duration
        else:
            return None, None
    else:
        return None, None

def find_center(sw_lat, sw_lng, ne_lat, ne_lng):
    center_lat = (sw_lat + ne_lat) / 2
    center_lng = (sw_lng + ne_lng) / 2
    return center_lat, center_lng

def generate_list(n, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")
    result = [0]
    random_values = np.random.rand(n - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    result.extend(rounded_values)

    return result

def crop_matrix(data, N):
    return [row[:N] for row in data[:N]]

import numpy as np

def generate_demand_list(n, m, target_sum):
    if n <= 1:
        raise ValueError("The length of the list must be greater than 1.")
    if m >= n:
        raise ValueError("M must be less than N.")
    if target_sum < 0:
        raise ValueError("The target sum must be non-negative.")

    random_values = np.random.rand(n - m - 1)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    zeros_list = [0] * m

    result = list(rounded_values) + zeros_list

    random.shuffle(result)

    result = [-1] + result

    return result
def replace_zeros_with_sum(input_list, target_sum):
    zero_positions = [i for i, value in enumerate(input_list) if value == 0]
    num_zeros = len(zero_positions)

    result = input_list.copy()

    if num_zeros == 0:
        raise ValueError("There are no zeros to replace in the list.")

    random_values = np.random.rand(num_zeros)
    random_values *= target_sum / np.sum(random_values)
    rounded_values = np.round(random_values, 2)
    difference = target_sum - np.sum(rounded_values)

    if len(rounded_values) > 0:
        rounded_values[-1] += difference

    for i, pos in enumerate(zero_positions):
        result[pos] = rounded_values[i]

    return result

def create_path_as_list(pairs_list):
    wanted_length = len(pairs_list)
    path = ''
    current_pair = next(pair for pair in pairs_list if pair[0] == 0)
    path += str(current_pair[0])

    next_start = current_pair[1]

    pairs_list.remove(current_pair)

    while len(path) < wanted_length:
        path += str(next_start)
        current_pair = next(pair for pair in pairs_list if pair[0] == next_start)
        pairs_list.remove(current_pair)
        next_start = current_pair[1]

    return path

def find_paths_vrb(edges, k):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == 0 and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])

    return paths

def repetitions(list1, list2):
  common_elements = set(list1) & set(list2)
  return (2*len(common_elements))/len(list1+list2)

def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [ ]:
def equal_paths(list1, list2):
  are_equal = sorted(list1) == sorted(list2)
  if are_equal:
    return 1
  else:
    return 0

In [ ]:
def create_cost_matrixes(df):
    real_distance_columns = [col for col in df.columns if col.startswith('Road Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_raod = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Road Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_raod[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Time to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_time = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Time to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_time[i][j] = float(distance)


    real_distance_columns = [col for col in df.columns if col.startswith('Real Distance to Point')]
    df_filtered = df[real_distance_columns]

    num_points = len(df_filtered)
    cost_real = [[0] * num_points for _ in range(num_points)]

    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                column_name = f'Real Distance to Point {j + 1}'
                distance = df_filtered.iloc[i][column_name]

                cost_real[i][j] = float(distance)

    return cost_raod, cost_time, cost_real

CRACOW VPTW

In [ ]:
def find_paths_vrb(edges, k, depot_one_more_time):
    def find_path_vrb(current_path):
        if len(paths) >= k:
            return

        last_node = current_path[-1]

        if last_node == depot_one_more_time and len(current_path) > 1:
            paths.append(current_path)
            return

        for edge in edges:
            if edge[0] == last_node:
                find_path_vrb(current_path + [edge[1]])

    paths = []
    find_path_vrb([0])
    return paths

def vrptw(number_of_clients, number_of_vehicles, cost_buffor, a, b, t, T,  file_idx, problem_idx):
    try:
      n = number_of_clients
      c = number_of_vehicles
      Q = 100
      M = 10000000

      cost_shiffted = crop_matrix(cost_buffor,n+1)
      cost_shiffted.append(cost_shiffted[0])
      iterator = 0
      for row in cost_shiffted:
        cost_shiffted[iterator].append(cost_shiffted[iterator][0])
        iterator += 1

      prob = pulp.LpProblem("VRPTW", pulp.LpMinimize)

      x = pulp.LpVariable.dicts("x", [(i,j,k) for i in range(n+1) for j in range(1, n+2) for k in range(c)], cat='Binary')
      s = pulp.LpVariable.dicts("s", [(i,k) for i in range(n+2) for k in range(c)], lowBound=0, cat='Continuous')

      prob += pulp.lpSum(cost_shiffted[i][j] * x[(i,j,k)] for i in range(n+1) for j in range(1, n+2) for k in range(c))

      for j in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for i in range(n+1) for k in range(c)) == 1

      for i in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for j in range(1, n+2) for k in range(c)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[i,n+1,k] for i in range(n+1)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[0,j,k] for j in range(1, n+2)) == 1

      for m in range(1, n+1):
        for k in range(c):
          prob += pulp.lpSum(x[(i,m,k)] for i in range(n+1) if i != m) == pulp.lpSum(x[(m,j,k)] for j in range(1,n+2) if j != m)

      for k in range(c):
        prob += x[0,n+1,k] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            if i == j:
              prob += x[(i,j,k)] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            prob += s[(i,k)] + t[i] + T[i][j] <= s[(j,k)] + M*(1 - x[(i,j,k)])

      for i in range(1, n+1):
        for k in range(c):
          prob += M*(pulp.lpSum(x[i,j,k] for j in range(1,n+2)) - 1) + a[i] <= s[(i,k)]

      for i in range(1, n+1):
        for k in range(c):
          prob += s[(i,k)] + t[i] <= b[i] + M*(1 - pulp.lpSum(x[(i,j,k)] for j in range(1,n+2)))

      prob.solve()

      print(pulp.value(prob.objective))

      all_uesd_edges = []
      paths = [[] for _ in range(c)]

      for k in range(c):
          for i in range(n + 1):
              for j in range(1, n + 2):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))

      result_paths = find_paths_vrb(all_uesd_edges, number_of_clients, number_of_clients+1)
      return pulp.value(prob.objective), result_paths, all_uesd_edges

    except Exception as e:
        print(f"error for delivery stops {number_of_clients} and vehicles {number_of_vehicles}")

        values_to_reload.append((number_of_clients, number_of_vehicles, file_idx, problem_idx))

        return None, None, None

def vrptw_error(number_of_clients, number_of_vehicles, cost_buffor, a, b, t, T, file_idx):
  try:
      n = number_of_clients
      c = number_of_vehicles
      Q = 100
      M = 10000000

      cost_shiffted = crop_matrix(cost_buffor,n+1)
      cost_shiffted.append(cost_shiffted[0])
      iterator = 0
      for row in cost_shiffted:
        cost_shiffted[iterator].append(cost_shiffted[iterator][0])
        iterator += 1

      prob = pulp.LpProblem("VRPTW", pulp.LpMinimize)

      x = pulp.LpVariable.dicts("x", [(i,j,k) for i in range(n+1) for j in range(1, n+2) for k in range(c)], cat='Binary')
      s = pulp.LpVariable.dicts("s", [(i,k) for i in range(n+2) for k in range(c)], lowBound=0, cat='Continuous')

      prob += pulp.lpSum(cost_shiffted[i][j] * x[(i,j,k)] for i in range(n+1) for j in range(1, n+2) for k in range(c))

      for j in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for i in range(n+1) for k in range(c)) == 1

      for i in range(1, n+1):
          prob += pulp.lpSum(x[(i,j,k)] for j in range(1, n+2) for k in range(c)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[i,n+1,k] for i in range(n+1)) == 1

      for k in range(c):
          prob += pulp.lpSum(x[0,j,k] for j in range(1, n+2)) == 1

      for m in range(1, n+1):
        for k in range(c):
          prob += pulp.lpSum(x[(i,m,k)] for i in range(n+1) if i != m) == pulp.lpSum(x[(m,j,k)] for j in range(1,n+2) if j != m)

      for k in range(c):
        prob += x[0,n+1,k] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            if i == j:
              prob += x[(i,j,k)] == 0

      for i in range(n+1):
        for j in range(1,n+2):
          for k in range(c):
            prob += s[(i,k)] + t[i] + T[i][j] <= s[(j,k)] + M*(1 - x[(i,j,k)])

      for i in range(1, n+1):
        for k in range(c):
          prob += M*(pulp.lpSum(x[i,j,k] for j in range(1,n+2)) - 1) + a[i] <= s[(i,k)]

      for i in range(1, n+1):
        for k in range(c):
          prob += s[(i,k)] + t[i] <= b[i] + M*(1 - pulp.lpSum(x[(i,j,k)] for j in range(1,n+2)))

      prob.solve()

      print(pulp.value(prob.objective))

      all_uesd_edges = []
      paths = [[] for _ in range(c)]

      for k in range(c):
          for i in range(n + 1):
              for j in range(1, n + 2):
                  if i != j and pulp.value(x[i,j,k]) == 1:
                      all_uesd_edges.append((i, j))
                      paths[k].append((i, j))

      result = find_paths_vrb(all_uesd_edges, number_of_clients, number_of_clients+1)
      return pulp.value(vrpsdp.objective), result_paths, all_uesd_edges

  except Exception as e:
      print(f"ONE MORE TIME ERROR")
      return None, None, None

In [ ]:
# files = ['30_clients_cracow_1', '30_clients_cracow_2']#, '30_clients_cracow_3.csv','30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']

['30_clients_cracow_1', '30_clients_cracow_2', '30_clients_cracow_3.csv', '30_clients_cracow_4.csv', '30_clients_cracow_5.csv', '30_clients_cracow_6.csv', '30_clients_cracow_7.csv', '30_clients_cracow_8.csv', '30_clients_cracow_9.csv', '30_clients_cracow_10.csv', '30_clients_cracow_11.csv', '30_clients_cracow_12.csv', '30_clients_cracow_13.csv', '30_clients_cracow_14.csv', '30_clients_cracow_15.csv', '30_clients_cracow_16.csv', '30_clients_cracow_17.csv', '30_clients_cracow_18.csv', '30_clients_cracow_19.csv', '30_clients_cracow_20.csv']


In [ ]:
import math
import random

objectiv_function_road = [[[] for _ in range(3)] for _ in range(1)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(1)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(1)]
all_paths_road = [[[] for _ in range(3)] for _ in range(1)]
all_paths_time = [[[] for _ in range(3)] for _ in range(1)]
all_paths_real = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(1)]

values_to_reload = []
vechicles = [1,2,3]
clients = [10]

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)
  service_time = 5
  median = np.median(cost_T)
  max_time = math.floor(number_of_clients * median + service_time*number_of_clients/number_of_vehicles) +1

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      print(f"calculate for {number_of_clients} clients and {number_of_vehicles} vechicles")
      median = np.median(cost_T)
      max_time = math.floor(number_of_clients * median + service_time*number_of_clients/number_of_vehicles) +1

      time_window_time = max_time/4

      t = [service_time] * (number_of_clients + 10)
      a = [0] * (number_of_clients + 1)
      b = [max_time] * (number_of_clients + 1)

      for client in range(number_of_clients + 1):
        a[client] = random.randint(int(median/2), int(max_time - service_time - median))
        b[client] = a[client] + time_window_time


      obj_road, paths_road, edges_road = vrptw(number_of_clients, number_of_vehicles, cost_R, a, b, t, cost_T, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = vrptw(number_of_clients, number_of_vehicles, cost_T,  a, b, t, cost_T, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = vrptw(number_of_clients, number_of_vehicles, cost_G,  a, b, t, cost_T, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

############# calculated_file 30_clients_cracow_1
calculate for 10 clients and 1 vechicles
44.900005306999994
104.99999999999999
14.3500063321
calculate for 10 clients and 2 vechicles
68.89999999999999
141.0
27.08
calculate for 10 clients and 3 vechicles
61.800000000000004
139.0
23.479999999999997
############# calculated_file 30_clients_cracow_2
calculate for 10 clients and 1 vechicles
48.39999999999999
116.0
27.659999999999997
calculate for 10 clients and 2 vechicles
45.50000000000001
107.0
24.769999999999996
calculate for 10 clients and 3 vechicles
44.699999999999996
114.0
24.679999999999996
############# calculated_file 30_clients_cracow_3.csv
calculate for 10 clients and 1 vechicles
56.10000000000001
131.0
33.69
calculate for 10 clients and 2 vechicles
50.800000000000004
117.0
28.66
calculate for 10 clients and 3 vechicles
46.4
119.0
26.979999999999997
############# calculated_file 30_clients_cracow_4.csv
calculate for 10 clients and 1 vechicles
28.0000195039955
63.00005012
13.950

KeyboardInterrupt: 

In [ ]:
percent = 0.62

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)
    service_time = 5
    median = np.median(cost_T)
    max_time = math.floor(number_of_clients/number_of_vehicles * median + service_time*number_of_clients/number_of_vehicles) +1
    t = [service_time] * (number_of_clients + 10)
    a = [0] * (number_of_clients + 1)
    b = [max_time] * (number_of_clients + 1)
    time_window_time = max_time/5

    for client in range(number_of_clients + 1):
      a[client] = random.randint(median/2, max_time - service_time - median)
      b[client] = a[client] + time_window_time

    obj_1, paths, edges = vrptw_error(number_of_clients, number_of_vehicles, cost_road,  a, b, t, cost_T, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = vrptw_error(number_of_clients, number_of_vehicles, cost_time,  a, b, t, cost_T, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = vrptw_error(number_of_clients, number_of_vehicles, cost_real,  a, b, t, cost_T, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(2):

      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))

In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(1)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(1)]

AVG_G = [[[] for _ in range(3)] for _ in range(1)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(1)]

AVG_T = [[[] for _ in range(3)] for _ in range(1)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_T_edges = [[[] for _ in range(3)] for _ in range(1)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_G_edges = [[[] for _ in range(3)] for _ in range(1)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

G_T_edges = [[[] for _ in range(3)] for _ in range(1)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_T_paths = [[[] for _ in range(3)] for _ in range(1)]
R_G_paths = [[[] for _ in range(3)] for _ in range(1)]
G_T_paths = [[[] for _ in range(3)] for _ in range(1)]



for vechicles_idx in range(3):
  for clients_idx in range(2):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])

In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")


AVG_R = [[33.095, 35.205, 38.78499999999999], [42.7889, 41.9628, 45.00279999999999]]
AVG_R_stdev = [[9.53427894494387, 9.718048929697773, 9.615262606918233], [8.905264790560695, 8.350619914712922, 8.57746285098339]]
AVG_T = [[76.95, 85.3, 96.6], [100.55, 102.9, 113.05]]
AVG_T_stdev = [[18.980187038066827, 19.31864384474231, 19.535608513685975], [19.946114909926695, 17.572421574728967, 18.222170562257396]]
AVG_G = [[15.358999999999998, 16.836499999999997, 19.192999999999998], [18.4255, 18.467499999999998, 20.592499999999998]]
AVG_G_stdev = [[2.3639266063057027, 2.4454412178582405, 2.6387669468901573], [2.6950481906637593, 2.7611100575674263, 3.0217095740656474]]
R_T_edges = [[0.7181818181818181, 0.8666666666666666, 0.8000000000000002], [0.725, 0.7970588235294117, 0.7749999999999999]]
R_T_edges_stdev = [[0.2071051954526276, 0.15898986690282427, 0.142671053776857], [0.1889527057228872, 0.1397909957448958, 0.13996802985937914]]
R_G_edges = [[0.5363636363636364, 0.5833333333333333, 0.603846

### EUROPE

In [ ]:
# ['30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4', '30_clients_europe_5', '30_clients_europe_7', '30_clients_europe_8', '30_clients_europe_6', '30_clients_europe_9', '30_clients_europe_10', '30_clients_europe_11', '30_clients_europe_12', '30_clients_europe_13', '30_clients_europe_14', '30_clients_europe_15', '30_clients_europe_16', '30_clients_europe_17', '30_clients_europe_18', '30_clients_europe_19', '30_clients_europe_20']

['30_clients_europe_1', '30_clients_europe_2', '30_clients_europe_3', '30_clients_europe_4', '30_clients_europe_5', '30_clients_europe_7', '30_clients_europe_8', '30_clients_europe_6', '30_clients_europe_9', '30_clients_europe_10', '30_clients_europe_11', '30_clients_europe_12', '30_clients_europe_13', '30_clients_europe_14', '30_clients_europe_15', '30_clients_europe_16', '30_clients_europe_17', '30_clients_europe_18', '30_clients_europe_19', '30_clients_europe_20']
20


In [ ]:
capacity_of_vehicle = 100

objectiv_function_road = [[[] for _ in range(3)] for _ in range(1)]
objectiv_function_time = [[[] for _ in range(3)] for _ in range(1)]
objectiv_function_real = [[[] for _ in range(3)] for _ in range(1)]
all_paths_road = [[[] for _ in range(3)] for _ in range(1)]
all_paths_time = [[[] for _ in range(3)] for _ in range(1)]
all_paths_real = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_road = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_time = [[[] for _ in range(3)] for _ in range(1)]
all_uesd_edges_real = [[[] for _ in range(3)] for _ in range(1)]

values_to_reload = []
vechicles = [1,2,3]
clients = [16]

for requested_file in files:
  print(f"############# calculated_file {requested_file}")
  file_idx = files.index(requested_file)
  df = pd.read_csv(os.path.join(output_dir, requested_file))
  df.fillna(10000000, inplace=True)
  cost_R, cost_T, cost_G = create_cost_matrixes(df)

  v_iterator = 0

  for number_of_vehicles in vechicles:
    c_iterator = 0

    for number_of_clients in clients:

      print(f"calculate for {number_of_clients} clients and {number_of_vehicles} vechicles")

      demand_of_client = generate_list(number_of_clients + 1, 0.7 * capacity_of_vehicle * number_of_vehicles)

      obj_road, paths_road, edges_road = cvrp(number_of_clients, number_of_vehicles, cost_R, demand_of_client, file_idx, "R")

      objectiv_function_road[c_iterator][v_iterator].append(obj_road)
      all_paths_road[c_iterator][v_iterator].append(paths_road)
      all_uesd_edges_road[c_iterator][v_iterator].append(edges_road)

      obj_time, paths_time, edges_time = cvrp(number_of_clients, number_of_vehicles, cost_T, demand_of_client, file_idx, "T")

      objectiv_function_time[c_iterator][v_iterator].append(obj_time)
      all_paths_time[c_iterator][v_iterator].append(paths_time)
      all_uesd_edges_time[c_iterator][v_iterator].append(edges_time)

      obj_real, paths_real, edges_real = cvrp(number_of_clients, number_of_vehicles, cost_G, demand_of_client, file_idx, "G")

      objectiv_function_real[c_iterator][v_iterator].append(obj_real)
      all_paths_real[c_iterator][v_iterator].append(paths_real)
      all_uesd_edges_real[c_iterator][v_iterator].append(edges_real)

      c_iterator += 1

    v_iterator += 1

print(values_to_reload)

############# calculated_file 30_clients_europe_1
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_2
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_3
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_4
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_5
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
calculate for 16 clients and 3 vechicles
############# calculated_file 30_clients_europe_7
calculate for 16 clients and 1 vechicles
calculate for 16 clients and 2 vechicles
cal

In [ ]:
percent = 0.62

while len(values_to_reload) != 0:
  values_to_reload_mew = []
  percent = percent - 0.02
  print(f"###### Failed roads {len(values_to_reload)}")
  for (number_of_clients, number_of_vehicles, file_idx, problem) in values_to_reload:
    print(f"{number_of_clients}, {number_of_vehicles}, {file_idx}, {problem}")
    df = pd.read_csv(os.path.join(output_dir, files[file_idx]))
    df.fillna(10000000, inplace=True)
    cost_road, cost_time, cost_real = create_cost_matrixes(df)
    deamnd = generate_list(number_of_clients + 1, percent * capacity_of_vehicle * number_of_vehicles)

    obj_1, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_road, deamnd, file_idx)
    if obj_1 != None:
        objectiv_function_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_1
        all_paths_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_road[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_2, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_time, deamnd, file_idx)
    if obj_2 != None:
        objectiv_function_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_2
        all_paths_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_time[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    obj_3, paths, edges = cvrp_error(number_of_clients, number_of_vehicles, cost_real, deamnd, file_idx)
    if obj_3 != None:
        objectiv_function_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = obj_3
        all_paths_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = paths
        all_uesd_edges_real[clients.index(number_of_clients)][vechicles.index(number_of_vehicles)][file_idx] = edges

    if obj_1 == None or obj_2 == None or obj_3 == None:
      values_to_reload_mew.append((number_of_clients, number_of_vehicles, file_idx, problem))

  values_to_reload = values_to_reload_mew


In [ ]:
R_T_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_G_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
G_T_edges_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_T_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]
R_G_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]
G_T_paths_preperation = [[[] for _ in range(3)] for _ in range(1)]


for file_idx in range(len(files)):
  for vechicles_idx in range(3):
    for clients_idx in range(1):

      R_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))
      R_G_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_road[clients_idx][vechicles_idx][file_idx], all_uesd_edges_real[clients_idx][vechicles_idx][file_idx]))
      G_T_edges_preperation[clients_idx][vechicles_idx].append(repetitions(all_uesd_edges_real[clients_idx][vechicles_idx][file_idx], all_uesd_edges_time[clients_idx][vechicles_idx][file_idx]))

      R_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))
      R_G_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_road[clients_idx][vechicles_idx][file_idx], all_paths_real[clients_idx][vechicles_idx][file_idx]))
      G_T_paths_preperation[clients_idx][vechicles_idx].append(equal_paths(all_paths_real[clients_idx][vechicles_idx][file_idx], all_paths_time[clients_idx][vechicles_idx][file_idx]))



In [ ]:
import numpy as np

AVG_R = [[[] for _ in range(3)] for _ in range(1)]
AVG_R_stdev = [[[] for _ in range(3)] for _ in range(1)]

AVG_G = [[[] for _ in range(3)] for _ in range(1)]
AVG_G_stdev = [[[] for _ in range(3)] for _ in range(1)]

AVG_T = [[[] for _ in range(3)] for _ in range(1)]
AVG_T_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_T_edges = [[[] for _ in range(3)] for _ in range(1)]
R_T_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_G_edges = [[[] for _ in range(3)] for _ in range(1)]
R_G_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

G_T_edges = [[[] for _ in range(3)] for _ in range(1)]
G_T_edges_stdev = [[[] for _ in range(3)] for _ in range(1)]

R_T_paths = [[[] for _ in range(3)] for _ in range(1)]
R_G_paths = [[[] for _ in range(3)] for _ in range(1)]
G_T_paths = [[[] for _ in range(3)] for _ in range(1)]


for vechicles_idx in range(3):
  for clients_idx in range(1):
     AVG_R[clients_idx][vechicles_idx] = np.mean(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_R_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_road[clients_idx][vechicles_idx])
     AVG_T[clients_idx][vechicles_idx] = np.mean(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_T_stdev [clients_idx][vechicles_idx] = np.std(objectiv_function_time[clients_idx][vechicles_idx])
     AVG_G[clients_idx][vechicles_idx] = np.mean(objectiv_function_real[clients_idx][vechicles_idx])
     AVG_G_stdev[clients_idx][vechicles_idx] = np.std(objectiv_function_real[clients_idx][vechicles_idx])

     R_T_edges[clients_idx][vechicles_idx] = np.mean(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_T_edges_stdev[clients_idx][vechicles_idx] = np.std(R_T_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges[clients_idx][vechicles_idx] = np.mean(R_G_edges_preperation[clients_idx][vechicles_idx])
     R_G_edges_stdev[clients_idx][vechicles_idx] = np.std(R_G_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges[clients_idx][vechicles_idx] = np.mean(G_T_edges_preperation[clients_idx][vechicles_idx])
     G_T_edges_stdev[clients_idx][vechicles_idx] = np.std(G_T_edges_preperation[clients_idx][vechicles_idx])

     R_T_paths[clients_idx][vechicles_idx] = sum(R_T_paths_preperation[clients_idx][vechicles_idx])
     R_G_paths[clients_idx][vechicles_idx] = sum(R_G_paths_preperation[clients_idx][vechicles_idx])
     G_T_paths[clients_idx][vechicles_idx] = sum(G_T_paths_preperation[clients_idx][vechicles_idx])


In [ ]:
print(f"AVG_R = {AVG_R}")
print(f"AVG_R_stdev = {AVG_R_stdev}")
print(f"AVG_T = {AVG_T}")
print(f"AVG_T_stdev = {AVG_T_stdev}")
print(f"AVG_G = {AVG_G}")
print(f"AVG_G_stdev = {AVG_G_stdev}")

print(f"R_T_edges = {R_T_edges}")
print(f"R_T_edges_stdev = {R_T_edges_stdev}")
print(f"R_G_edges = {R_G_edges}")
print(f"R_G_edges_stdev = {R_G_edges_stdev}")
print(f"G_T_edges = {G_T_edges}")
print(f"G_T_edges_stdev = {G_T_edges_stdev}")

print(f"R_T_paths = {R_T_paths}")
print(f"R_G_paths = {R_G_paths}")
print(f"G_T_paths = {G_T_paths}")


AVG_R = [[4066.2349999999997, 4563.99, 5075.629999999988]]
AVG_R_stdev = [[476.2125610218614, 490.3885438099067, 592.6652647996107]]
AVG_T = [[2953.75, 3272.9500000000044, 3584.95]]
AVG_T_stdev = [[326.23976995455354, 319.0369061723157, 365.9783429384859]]
AVG_G = [[2991.7264999999998, 3371.71, 3764.744]]
AVG_G_stdev = [[338.48557842654094, 339.10083528354807, 393.160859387096]]
R_T_edges = [[0.5264705882352941, 0.26240117014547754, 0.47036852863478246]]
R_T_edges_stdev = [[0.41655823271863773, 0.2027142883824963, 0.32447731137947583]]
R_G_edges = [[0.4740239204186073, 0.5336390470166561, 0.3953702619027696]]
R_G_edges_stdev = [[0.37449521529859525, 0.2764611084057748, 0.2427632142240859]]
G_T_edges = [[0.4271836007130124, 0.26703036053130935, 0.47161349319244056]]
G_T_edges_stdev = [[0.38576968122863114, 0.23634026334701153, 0.29031373460380044]]
R_T_paths = [[5, 0, 4]]
R_G_paths = [[2, 2, 1]]
G_T_paths = [[2, 0, 2]]


### RESULTS

In [ ]:
for client in clients:
  for vechicle in vechicles:
    df.loc[iterator] = [topology, client, vechicle,
                        str(AVG_R[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_R_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_G[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_G_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(AVG_T[clients.index(client)][vechicles.index(vechicle)]) + " ± " + str(plus_minus(AVG_T_stdev[clients.index(client)][vechicles.index(vechicle)])),
                        str(R_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(R_G_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(R_G_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        str(G_T_edges[clients.index(client)][vechicles.index(vechicle)]) + "% ± " + str(plus_minus(G_T_edges_stdev[clients.index(client)][vechicles.index(vechicle)])) +"%",
                        R_T_paths[clients.index(client)][vechicles.index(vechicle)], R_G_paths[clients.index(client)][vechicles.index(vechicle)], G_T_paths[clients.index(client)][vechicles.index(vechicle)]
                        ]
    iterator += 1


In [ ]:
| Topology | No. clients | No. vehicles | Road distance matrix | Geographical distance matrix | Time distance matrix |
|----------|-------------|--------------|----------------------|-----------------------------|----------------------|
| Krakow   | 10          | 1            | 52.0 ± 7.252         | 25.031 ± 1.957              | 113.15 ± 13.365      |
| Krakow   | 10          | 2            | 35.265 ± 4.576       | 16.887 ± 1.121              | 85.3 ± 9.042         |
| Krakow   | 10          | 3            | 38.83 ± 4.533        | 19.193 ± 1.235              | 96.6 ± 9.143         |
| Krakow   | 15          | 1            | 68.579 ± 5.949       | 29.478 ± 1.892              | 150.55 ± 11.256      |
| Krakow   | 15          | 2            | 41.973 ± 3.888       | 18.608 ± 1.284              | 102.8 ± 8.269        |
| Krakow   | 15          | 3            | 45.723 ± 3.859       | 20.694 ± 1.441              | 113.05 ± 8.63        |
| Europe   | 10          | 1            | 3989.12 ± 377.172    | 2968.462 ± 268.147          | 2711.75 ± 246.468    |
| Europe   | 10          | 2            | 4192.88 ± 360.228    | 3077.568 ± 278.386          | 2882.051 ± 220.044   |
| Europe   | 10          | 3            | 4371.225 ± 321.881   | 3251.014 ± 213.613          | 2964.85 ± 210.562    |
| Europe   | 15          | 1            | 4918.775 ± 318.203   | 3663.907 ± 223.45           | 3438.55 ± 190.094    |
| Europe   | 15          | 2            | 5364.451 ± 426.981   | 3987.567 ± 292.043          | 3719.2 ± 249.432     |
| Europe   | 15          | 3            | 5326.411 ± 285.186   | 3952.688 ± 205.338          | 3695.85 ± 183.214    |


In [ ]:
| Topology | No. clients | No. vehicles | R/T e. [\%]       | R/G e. [\%]       | G/T e. [\%]       | R/T p. | R/G p. | G/T p. |
|----------|-------------|--------------|-------------------|-------------------|-------------------|--------|--------|--------|
| Krakow   | 10          | 1            | 73.20 ± 13.73      | 54.81 ± 15.66      | 54.94 ± 14.61      | 9      | 6      | 4      |
| Krakow   | 10          | 2            | 64.51 ± 18.48      | 34.76 ± 17.08      | 41.61 ± 16.49      | 7      | 3      | 4      |
| Krakow   | 10          | 3            | 57.67 ± 14.13      | 55.86 ± 13.90      | 47.41 ± 15.53      | 3      | 2      | 4      |
| Krakow   | 15          | 1            | 64.50 ± 13.37      | 41.44 ± 10.93      | 41.67 ± 8.95       | 3      | 0      | 0      |
| Krakow   | 15          | 2            | 42.23 ± 12.73      | 36.00 ± 12.74      | 35.59 ± 13.01      | 2      | 1      | 1      |
| Krakow   | 15          | 3            | 38.84 ± 14.77      | 42.38 ± 14.52      | 36.74 ± 14.51      | 3      | 1      | 1      |
| Europe   | 10          | 1            | 84.91 ± 6.97       | 72.96 ± 13.32      | 72.42 ± 13.33      | 9      | 8      | 8      |
| Europe   | 10          | 2            | 62.78 ± 16.33      | 62.54 ± 12.34      | 44.08 ± 15.27      | 7      | 7      | 5      |
| Europe   | 10          | 3            | 77.30 ± 9.30       | 75.10 ± 8.21       | 64.86 ± 11.29      | 5      | 5      | 4      |
| Europe   | 15          | 1            | 76.60 ± 12.57      | 68.11 ± 12.05      | 70.94 ± 10.04      | 5      | 4      | 4      |
| Europe   | 15          | 2            | 70.41 ± 13.60      | 65.83 ± 13.61      | 65.68 ± 13.83      | 6      | 4      | 2      |
| Europe   | 15          | 3            | 67.45 ± 13.65      | 63.73 ± 11.35      | 60.77 ± 12.53      | 4      | 2      | 1      |
